In [1]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\jump3\Desktop\Senior_Design_Project\Py_Modules\YOLOv8


In [6]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

#Disable piece of shit API
os.environ['WANDB_MODE'] = 'disabled'
from ultralytics import settings
settings.update({"wandb": False})

Ultralytics YOLOv8.2.92  Python-3.11.5 torch-2.4.1+cpu CPU (Intel Core(TM) i9-10885H 2.40GHz)
Setup complete  (16 CPUs, 31.8 GB RAM, 635.5/933.4 GB disk)


In [15]:
# Load a model
model_path = "loadable_models/testing/TEST_LoadModPT.onnx"
model = YOLO(model_path,task='classify')


In [12]:
img_path = 'datasets/TEST_example.jpg'
model(img_path)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: images for the following indices
 index: 2 Got: 640 Expected: 512
 index: 3 Got: 640 Expected: 384
 Please fix either the inputs/outputs or the model.

In [212]:
import onnxruntime

opt_session = onnxruntime.SessionOptions()
opt_session.enable_mem_pattern = False
opt_session.enable_cpu_mem_arena = False
opt_session.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_DISABLE_ALL

EP_list = ['CUDAExecutionProvider', 'CPUExecutionProvider']

ort_session = onnxruntime.InferenceSession(model_path, providers=EP_list)

model_inputs = ort_session.get_inputs()
input_names = [model_inputs[i].name for i in range(len(model_inputs))]
# print(input_names)
input_shape = model_inputs[0].shape
print(input_shape)

model_output = ort_session.get_outputs()
output_names = [model_output[i].name for i in range(len(model_output))]
print(output_names)

[1, 3, 512, 384]
['output0']


c:\Users\jump3\AppData\Local\Programs\Python\Python311\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [22]:
import cv2
import numpy as np
from PIL import Image

image = cv2.imread(img_path)
image_height, image_width = image.shape[:2]
# Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

input_height, input_width = input_shape[2:]
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized = cv2.resize(image_rgb, (input_width, input_height))

# Scale input pixel value to 0 to 1
input_image = resized / 255.0
input_image = input_image.transpose(2,0,1)
input_tensor = input_image[np.newaxis, :, :, :].astype(np.float32)
input_tensor.shape

(1, 3, 512, 384)

In [201]:
outputs = ort_session.run(output_names, {input_names[0]: input_tensor})[0]

predictions = np.squeeze(outputs).T
conf_thresold = 0.8
# Filter out object confidence scores below threshold
scores = np.max(predictions[:, 4:], axis=1)
predictions = predictions[scores > conf_thresold, :]
scores = scores[scores > conf_thresold]
print(scores)
print(predictions)


[    0.87847      0.9798     0.97593      0.8625     0.94462     0.93551]
[[     197.16      228.85      178.27      321.37   0.0028993   0.0060863     0.87847    0.013229   0.0062939]
 [        198      229.58      179.92      324.52   0.0031983    0.012272      0.9798    0.013839   0.0073553]
 [     197.79      228.81      179.29      322.68   0.0036263   0.0086037     0.97593    0.016285   0.0037619]
 [      198.6      229.56      179.73      322.39   0.0031632   0.0057328      0.8625   0.0086091   0.0077896]
 [     198.67      228.46      180.34      321.29    0.001851   0.0074157     0.94462    0.011874   0.0059485]
 [     197.87      228.15      179.03      322.13   0.0025014   0.0086324     0.93551   0.0094804    0.002876]]


In [202]:
# Get the class with the highest confidence
class_ids = np.argmax(predictions[:, 4:], axis=1)
print(class_ids)

# Get bounding boxes for each object
boxes = predictions[:, :4]

#rescale box
input_shape = np.array([input_width, input_height, input_width, input_height])
boxes = np.divide(boxes, input_shape, dtype=np.float32)
boxes *= np.array([image_width, image_height, image_width, image_height])
boxes = boxes.astype(np.int32)
print(boxes)

[2 2 2 2 2 2]
[[262 171 237 241]
 [264 172 239 243]
 [263 171 239 242]
 [264 172 239 241]
 [264 171 240 240]
 [263 171 238 241]]


In [210]:
def simular_boxes(box1,box2,tolerance=10):
    # print('a',box1,box2)
    check = abs(box1[0] - box2[0])<= tolerance
    check *= abs(box1[1] - box2[1])<= tolerance
    check *= abs(box1[2] - box2[2])<= tolerance
    check *= abs(box1[3] - box2[3])<= tolerance
    return check

def reduce_list(work):
    working=work.tolist()
    # print(working)
    cnt=0;cnt2=cnt+1
    while cnt<len(working):
        while cnt2<len(working):
            if simular_boxes(   working[cnt], working[cnt2]   ):
                # print(f'ye: {cnt},{cnt2}')
                working.pop(cnt2)
            else: cnt2+=1
        cnt+=1;cnt2=cnt+1
    return working

def find_list_in_LoL(LoL,targ):
    for i, row in enumerate(LoL):
        if np.array_equal(row, targ):
            return i
    return -1

def reduce_detections(predictions,classes=['Cardboard','glass','metal','paper','plas']):
    class_ids = np.argmax(predictions[:, 4:], axis=1)
    # print(class_ids)

    # Get bounding boxes for each object
    boxes = predictions[:, :4]

    #rescale box
    input_shape = np.array([input_width, input_height, input_width, input_height])
    boxes = np.divide(boxes, input_shape, dtype=np.float32)
    boxes *= np.array([image_width, image_height, image_width, image_height])
    boxes = boxes.astype(np.int32)
    
    reduced = [ find_list_in_LoL(boxes,non_sim) for non_sim in reduce_list(boxes)  ]
    results=[]
    for ele in reduced:
        results.append(   [  classes[class_ids[ele]], [boxes[ele][:2].tolist(),boxes[ele][2:].tolist()]  ]   )
    return results

#-------------------------
reduce_detections(predictions)

[['metal', [[262, 171], [237, 241]]]]

In [206]:
reduce_detections(predictions)

[0]

In [92]:
boxes[0]

array([262, 171, 237, 241])

In [200]:
ex_class_ids = class_ids[:]
ex_class_ids[-1]=3

num_class=predictions.shape[1]-4
num_preds = len(ex_class_ids)
bests=[];idx=[]
for cls in range(num_class):
    idx.append( [ [scores[i],i] for i in range(num_preds) if cls==ex_class_ids[i] ] )
print(idx)
for ele in idx:
    if ele: bests.append(max(ele, key = lambda i : i[0])[1])
    else: bests.append(-1)
print(bests)
conclusion = [ [boxes[i],ex_class_ids[i]] for i in bests if i!=-1  ]
conclusion


[[], [], [[0.8784683, 0], [0.9798023, 1], [0.9759264, 2], [0.86249614, 3], [0.9446247, 4]], [[0.9355068, 5]], []]
[-1, -1, 1, 5, -1]


[[array([264, 172, 239, 243]), 2], [array([263, 171, 238, 241]), 3]]

So can either pursue ONE object or ONE object per high class guess, need to break into simularity

In [102]:
example_boxes = np.array(
    [
    [100,200,300,400],
    [101,202,303,404],
    [102,201,302,401],
    
    [200,100,250,150],
    [205,105,255,155],
    [203,103,253,153],
    [102,201,302,400]
    ]
)
example_boxes

array([[100, 200, 300, 400],
       [101, 202, 303, 404],
       [102, 201, 302, 401],
       [200, 100, 250, 150],
       [205, 105, 255, 155],
       [203, 103, 253, 153]])

In [156]:
def simular_boxes(box1,box2,tolerance=10):
    # print('a',box1,box2)
    check = abs(box1[0] - box2[0])<= tolerance
    check *= abs(box1[1] - box2[1])<= tolerance
    check *= abs(box1[2] - box2[2])<= tolerance
    check *= abs(box1[3] - box2[3])<= tolerance
    return check

def reduce_list(work):
    working=work.tolist()
    # print(working)
    cnt=0;cnt2=cnt+1
    while cnt<len(working):
        while cnt2<len(working):
            if simular_boxes(   working[cnt], working[cnt2]   ):
                # print(f'ye: {cnt},{cnt2}')
                working.pop(cnt2)
            else: cnt2+=1
        cnt+=1;cnt2=cnt+1
    return working
    
reduce_list(example_boxes)

[[100, 200, 300, 400], [200, 100, 250, 150]]

In [173]:
reduce_list(boxes)

[[262, 171, 237, 241]]

In [193]:
def find_list_in_LoL(LoL,targ):
    for i, row in enumerate(LoL):
        if np.array_equal(row, targ):
            return i
    return -1

cop_box = boxes.copy()

a=reduce_list(cop_box)
np.random.shuffle(cop_box)
print(cop_box)
print(a[0])

find_list_in_LoL(cop_box,a[0])

[[264 172 239 243]
 [264 171 240 240]
 [262 171 237 241]
 [263 171 238 241]
 [264 172 239 241]
 [263 171 239 242]]
[262, 171, 237, 241]


2

In [198]:
[ find_list_in_LoL(example_boxes,non_sim) for non_sim in reduce_list(example_boxes)  ]

[0, 3]